In [1]:
import numpy as np 

grid = np.arange(13)
a,b = np.meshgrid(grid,grid)

print(grid)
print(a)
print(b)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
[[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12]]
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  1  1  1  1  1  1  1  1  1  1  1  1]
 [ 2  2  2  2  2  2  2  2  2  2  2  2  2]
 [ 3  3  3  3  3  3  3  3  3  3  3  3  3]
 [ 4  4  4  4  4  4  4  4  4  4  4  4  4]
 [ 5  5  5  5  5  5  5  5  5  5  5  5  5]
 [ 6  6  6  6  6  6  6  6  6  6  6  6  6]
 [ 7  7  7  7  7  7  7  7  7  7  7  7  7]
 [ 8  8  8  8  8  8  8  8  8  8  8  8  8]
 [ 9  9  9  9  9  9  9  9  9  9  9

In [ ]:
import torch

grid_t = torch.arange(13)
c,d = torch.meshgrid(grid_t,grid_t)
print(c)
print(d)

In [ ]:
import torch

x_offset = torch.Tensor(a).view(-1,1)
x_offset

In [ ]:
y_offset = torch.Tensor(b).view(-1,1)
y_offset

In [ ]:
x_y_offset = torch.cat((x_offset,y_offset),1)
x_y_offset

In [ ]:
x_y_offset = torch.cat((x_offset, y_offset), 1).repeat(1, 507).view(-1, 2).unsqueeze(0)
x_y_offset

In [8]:
from darknet53 import Darknet53
import torch
from utils import predict_transform

dummy_input = torch.rand((1,3,416,416))

model = Darknet53(20)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
dummy_input = dummy_input.to(device)

output = model(dummy_input)

anchors = [10, 13, 16, 30, 33, 23, 30, 61, 62, 45, 59, 119, 116, 90, 156, 198, 373, 326]
anchors = [(anchors[i], anchors[i + 1]) for i in range(0, len(anchors), 2)]

mask = [[6,7,8],[3,4,5],[0,1,2]]

preds = []
for i,op in enumerate(output):
    sele_anchors = [anchors[j] for j in mask[i]]
    x = predict_transform(op,416,sele_anchors,20,device)
    preds.append(x)

pred = torch.cat(preds,dim=1)
print(pred.shape)





torch.Size([1, 507, 25])
torch.Size([1, 2028, 25])
torch.Size([1, 8112, 25])
torch.Size([1, 10647, 25])


In [2]:
from darknet import *
from utils import *
import torch
import cv2

model = Darknet("yolov3.cfg")
model.load_weights("yolov3.weights")

dummy_input = torch.rand((1,3,608,608))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
dummy_input = dummy_input.to(device)

img = cv2.imread("person.jpg")
img = prep_image(img,(608,608))
img = img.to(device)
pred = model(img,device)

pred = write_results(pred,0.5,80)

print(pred)


tensor([[  0.0000, 179.6514, 203.2439, 259.9902, 456.2253,   0.9998,   1.0000,
           0.0000],
        [  0.0000,  59.8457, 353.9190, 188.0535, 435.4765,   0.9996,   0.9954,
          16.0000],
        [  0.0000, 379.4738, 235.1684, 574.0648, 443.0357,   0.9991,   0.9678,
          17.0000]], device='cuda:0')
